# Семинар 7: Actor-Critic


## Майнор ВШЭ, 7.03.2019

На этом семинаре будем рассматривать простой эксперимент с алгоритмом Actor-Critic.

Рассмотрим задачу с маятиником - здесь количество действий не ограничено!

![Маятник](https://cdn-images-1.medium.com/max/1600/1*J_oEx0kpBpwXoVmRytn6qg.gif)

Здесь невозможно правильно определить функцию $Q(s,a)$.

![Модель Actor-Critic](https://cdn-images-1.medium.com/max/1600/1*-GfRVLWhcuSYhG25rN0IbA.png)


Модель актор-критик имеет два отдельных аппроксиматора: первый используется для предсказания того, какое действие предпринять, учитывая текущее состояние среды, и второй - чтобы найти значение полезности действия/состояния.

### Задание 1. Модель актора

Начнем с импортирования всех необходимых библиотек.

In [1]:
import gym
import numpy as np 
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import Add, Multiply
from keras.optimizers import Adam
import keras.backend as K

import tensorflow as tf

import random
from collections import deque

Using TensorFlow backend.


Тестировать нашу модель будем на простой задаче с перевернутым маятником.

In [5]:
## Создаем окружение Pendulum - смотрим как оно выглядит
env = gym.make("Pendulum-v0")

In [6]:
learning_rate = 0.001

memory = deque(maxlen=2000)

Начнем с определения модели актера. Цель модели состоит в том, чтобы, учитывая текущее состояние среды, определить наилучшее действие. Для начала будем использовать несколько полносвязных слоев, отображающих наблюдение среды в точку в пространстве среды.

In [7]:
def create_actor_model(env):
    state_input = Input(shape=env.observation_space.shape)
    h1 = Dense(24, activation='relu')(state_input)
    h2 = Dense(48, activation='relu')(h1)
    h3 = Dense(24, activation='relu')(h2)
    output = Dense(env.action_space.shape[0], activation='relu')(h3)

    model = Model(input=state_input, output=output)
    adam  = Adam(lr=0.001)
    model.compile(loss="mse", optimizer=adam)
    return state_input, model

Далее мы хотим определить, какое изменение параметров (в модели актора) приведет к наибольшему увеличению значения Q (предсказанному моделью критика).

In [8]:
actor_state_input, actor_model = create_actor_model(env)
_, target_actor_model = create_actor_model(env)

actor_critic_grad = tf.placeholder(tf.float32, 
    [None, env.action_space.shape[0]]) # where we will feed de/dC (from critic)

actor_model_weights = actor_model.trainable_weights
actor_grads = tf.gradients(actor_model.output, 
    actor_model_weights, -actor_critic_grad) # dC/dA (from actor)
grads = zip(actor_grads, actor_model_weights)
optimize = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


### Задание 2. Модель критика

Критик нужен для того, чтобы по состоянию окружающей среды и совершенному действию в качестве входных данных рассчитать соответствующую оценку. Построим его модель с помощью нескольких полвносвязных слоев с одним объединением перед окончательным предсказанием Q-значения.

In [9]:
def create_critic_model(env):
    state_input = Input(shape=env.observation_space.shape)
    state_h1 = Dense(24, activation='relu')(state_input)
    state_h2 = Dense(48)(state_h1)

    action_input = Input(shape=env.action_space.shape)
    action_h1    = Dense(48)(action_input)

    merged    = Add()([state_h2, action_h1])
    merged_h1 = Dense(24, activation='relu')(merged)
    output = Dense(1, activation='relu')(merged_h1)
    model  = Model(input=[state_input,action_input], output=output)

    adam  = Adam(lr=0.001)
    model.compile(loss="mse", optimizer=adam)
    return state_input, action_input, model

Нам нужны ссылки как на входы действий, так и на входы по состоянию, поскольку нам нужно использовать их при обновлении для сети актора.

In [10]:
critic_state_input, critic_action_input, critic_model = create_critic_model(env)
_, _, target_critic_model = create_critic_model(env)

critic_grads = tf.gradients(critic_model.output, critic_action_input) # where we calcaulte de/dC for feeding above

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [11]:
sess = tf.Session()
K.set_session(sess)
# Initialize for later gradient calculations
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


### Задание 3. Определяем метод обучения модели

Теперь определим метод обучения по памяти. Мы просто находим дисконтированное будущее вознаграждение и обучаемся на этом. Обучение проводим на паре состояние/действие и используем target_critic_model для прогнозирования будущей награды. Для актора мы уже определили как градиенты будут работать в сети, и теперь просто должны вызвать их с действиями и состояниями, которые берем из памяти.

In [12]:
gamma = .95

def train(sess, memory):
    batch_size = 32
    if len(memory) < batch_size:
        return
    rewards = []
    samples = random.sample(memory, batch_size)
  
    
    for sample in samples:
        cur_state, action, reward, new_state, _ = sample
        predicted_action = actor_model.predict(cur_state)
        grads = sess.run(critic_grads, feed_dict={
            critic_state_input:  cur_state,
            critic_action_input: predicted_action
        })[0]

        sess.run(optimize, feed_dict={
            actor_state_input: cur_state,
            actor_critic_grad: grads
        })
        
    for sample in samples:
        cur_state, action, reward, new_state, done = sample
        if not done:
            target_action = target_actor_model.predict(new_state)
            future_reward = target_critic_model.predict(
                [new_state, target_action])[0][0]
            reward += gamma * future_reward
        critic_model.fit([cur_state, action], reward, verbose=0)

В этой модели мы используем целевую сеть для улучшения сходимости. Мы должны обновлять ее веса на каждом временном шаге. Однако мы делаем это с меньшей частотой ,которая задается параметром $\tau$. Проделываем эту операцию как для актера так и для критика.

In [17]:
tau   = .125

def update_target():
    actor_model_weights  = actor_model.get_weights()
    actor_target_weights = target_actor_model.get_weights()

    for i in range(len(actor_target_weights)):
        actor_target_weights[i] = (1-tau)*actor_model_weights[i]+tau*actor_target_weights[i]
    target_actor_model.set_weights(actor_target_weights)


    critic_model_weights  = critic_model.get_weights()
    critic_target_weights = target_critic_model.get_weights()

    for i in range(len(critic_target_weights)):
        critic_target_weights[i] = (1-tau)*critic_model_weights[i]+tau*critic_target_weights[i]
    target_critic_model.set_weights(critic_target_weights)

### Задание 4. Проводим эксперименты.

Записываем полный цикл обучения: запоминиаем статистику и проводим на ней обучение.

In [18]:
num_trials = 10000
trial_len  = 500
epsilon = 1.0
epsilon_decay = .995

cur_state = env.reset()
action = env.action_space.sample()
while True:
    env.render()
    cur_state = cur_state.reshape((1, env.observation_space.shape[0]))
    
    epsilon *= epsilon_decay
    ### Реализуем epsilon-жадную стратегию
    if np.random.random() < epsilon:
        action =  env.action_space.sample()
    else:
        action = actor_model.predict(cur_state)
    
    
    action = action.reshape((1, env.action_space.shape[0]))

    new_state, reward, done, _ = env.step(action)
    new_state = new_state.reshape((1, env.observation_space.shape[0]))

    ### Сохраняем в память данные
    memory.append([cur_state, action, reward, new_state, done])
    
    train(sess, memory)
    
    update_target()

    cur_state = new_state

KeyboardInterrupt: 

Постройте график вознаграждения в зависимости от номера эпизода.